# 1. Definición de parámetros, diccionarios generales
##### Se inicializan parámetros y diccionarios que se usarán posteriormente, así como se declaran los archivos de donde se recoge la información.

In [ ]:
# Files For regressors Do Not Touch
regressors_df = {}
macros_df = {}
df = None

# About time and seasonality
dias_prediccion = 124
s_w = range(4, 8)
s_m = range(4, 7)
s_y = range(6, 14)


# Model Nature
_model = "linear"
_modality = "additive"


# Files Path
kpi_series = {"kpi_name": "motos_ios",
              "kpi_path": 'motos_ios.csv'}
macros = {"covid_evolution": 'covid_evolution.csv'}



# 2. Importar los módulos
##### Se importan los módulos, o alguna funcionalidad concreta de los mismos, que se usarán en el código de forecasting. Se setea la seed a un valor concreto para facilitar el poder replicar los resultados de ser necesario

In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from datetime import timedelta
import datetime
import warnings
import logging
from pyswarm import pso
import argparse

import itertools
import multiprocessing
from contextlib import contextmanager

import time
import warnings

warnings.filterwarnings('ignore')

# Turn off fbprophet stdout logger
logging.getLogger('fbprophet').setLevel(logging.ERROR)

np.random.seed(1234)


# 3. Declaración de funciones
##### Se declaran las funciones que se usarán durante todo el proceso. En especial se declara la función train_model en la que se usan los parámetros de entrada, así como otros parámetros para terminar de configurar el modelo.

##### Se destacan las funciones de set_regressors_df y set_macros_df, que sirven para pasar otros inputs para que lo tenga en cuenta el modelo (pueden ser los casos de covid, budget de campañas de TV...). Para adaptar este punto puede ser necesario contar con ayuda para clarificar algún punto.

In [ ]:

def set_regressors_df(regressors):
    global regressors_df

    for metric_name, metric_path in regressors.items():
        df_tmp = pd.read_csv(metric_path, sep=';')
        df_tmp.columns = ["ds", metric_name]
        df_tmp['ds'] = pd.to_datetime(df_tmp['ds'])
        regressors_df.update({metric_name: df_tmp})


def set_macros_df(macros):
    global macros_df
    for metric_name, metric_path in macros.items():
        df_tmp = pd.read_csv(metric_path, sep=';')
        df_tmp.columns = ["ds", metric_name]
        df_tmp['ds'] = pd.to_datetime(df_tmp['ds'])
        macros_df.update({metric_name: df_tmp})


def set_df(kpi_series):
    global df

    df = pd.read_csv(kpi_series["kpi_path"], sep=';')
    df.columns = ["ds", "y"]
    df['ds'] = pd.to_datetime(df['ds'])

    for metric_df in regressors_df.values():
        df = pd.merge(df, metric_df, how='left', on='ds')

    df = df.fillna(0)
    df['cap'] = df.y.quantile(.95)
    df['floor'] = df.y.quantile(.5)


def set_holidays():
    # TODO: future fuction to set holidays
    pass


def pre_set(kpi_series, regressors=None, macros=None, holidays=None, model=_model, modality=_modality):
    """
    Function to set the df for the prophet model training
    :param kpi_series: {"kpi_name": "path/to/kpi.csv"}
    :param regressors: dictionary = {"metric_name": "path/to/file.csv"}
    :return:
    """
    if regressors:
        set_regressors_df(regressors)

    if macros:
        set_regressors_df(macros)

    global df

    print(df)

    set_df(kpi_series)

    print(df.head())

    global _model, _modality, _holidays
    _model = model
    _modality = modality
    _holidays = holidays


@contextmanager
def poolcontext(*args, **kwargs):
    pool = multiprocessing.Pool(*args, **kwargs)
    yield pool
    pool.terminate()


def pso_fixed_time_efect(s, j, k):
    """
    s = params[0]
    j = params[1]
    k = params[2]
    """

    ub = [.99, .99, 20.5]
    lb = [.01, 0.01, 0.05]
    args = (s, j, k)

    xopt, fopt = pso(train_model, lb, ub,
                     maxiter=5, swarmsize=10,
                     minfunc=1, minstep=.1,
                     args=args, debug=False)

    return {'season_s': s, 'season_j': j, 'season_k': k, 'xopt': xopt, 'fopt': fopt}


def run(s_w, s_m, s_y):

    # Generate a list of tuples where each tuple is a combination of parameters.
    # The list will contain all possible combinations of parameters.
    paramlist = itertools.product(s_w, s_m, s_y)

    # Generate processes equal to the number of cores
    with poolcontext(processes= multiprocessing.cpu_count()-1) as pool:
        results = pool.starmap(pso_fixed_time_efect, paramlist)

    results = pd.DataFrame(results)

    return results


def train_model(x, *args):
    x0 = x[0]
    x1 = x[1]
    x2 = x[2]

    global df, regressors_df, macros_df

    s_w, s_m, s_y = args

    m = Prophet(growth=_model, seasonality_mode=_modality,
                holidays_prior_scale=x2,
                changepoint_range=x1,
                changepoint_prior_scale=x0)

    # Specifying seasonality:
    if regressors_df:
        for metric_name in regressors_df.keys():
            m.add_regressor(metric_name)

    if macros_df:
        for metric_name in macros_df.keys():
            m.add_regressor(metric_name)

    
    m.add_seasonality(name='weekly', period=7, fourier_order=s_w)
    m.add_seasonality(name='monthly', period=30.5, fourier_order=s_m)
    m.add_seasonality(name='yearly', period=365.25, fourier_order=s_y)

    m.add_country_holidays(country_name='ES')
    m = m.fit(df)

    future = m.make_future_dataframe(dias_prediccion)

    future['ds'] = pd.to_datetime(future['ds'])

    for metric_df in regressors_df.values():
        future = pd.merge(future, metric_df, how='left', on='ds')

    future['cap'] = df.y.quantile(.95)
    future['floor'] = df.y.quantile(.5)

    df_cv = cross_validation(m, initial=pd.Timedelta('365.25 days'),
                             horizon=pd.Timedelta(str(dias_prediccion) + ' days'))

    df_p = performance_metrics(df_cv)


    return df_p.rmse.mean()


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("-f", dest="input_path", required=True,
                        help="input path with the .cql files", metavar="--file",
                        type=argparse.FileType('r'), nargs='+')
    args = parser.parse_args()

    for f in args.file:
        print(f)






# 4. Cálculo de los hiperparámetros
##### Se itera hasta encontrar los mejores hiperparámetros. Este proceso puede durar algunas horas, y hace un uso extensivo de la CPU del ordenador. Este proceso no hace falta recrearlo cada vez que se quiere actualizar el output del forecasting, se puede actualizar, por ejemplo, una vez al mes, o al Q.
##### Tras este proceso y en las siguientes cajas, se muestra la duración del proceso en horas, así como se importa el resultado del mismo por si se quiere verificar que relamente los hiperparámetros elegidos son los óptimos.

In [ ]:
start_time = time.time()

pre_set(kpi_series, regressors=None, macros = macros, holidays = None, model="linear", modality="additive")

results = run(s_w, s_m, s_y)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print((time.time() - start_time)/3600.0)

In [ ]:
min_df = results[results.fopt == results.fopt.min()]

In [ ]:
min_df.to_csv('motos_android_param.csv', sep=';', index=False)

In [ ]:
min_df

# 5. Proceso de forecasting
##### Se hace el forecasting usando los parámetros, así como los regresores declarados. Como resultado, hay algunas visualizaciones en las que se puede apreciar el forecast en diferentes granularidades, se hace una validación cruzada, y se puede ver el mape. Este proceso se puede ejecutar diariamente, para ir actualizando los registros, ya que se forecastea X días en el futuro.

##### También se exportan estos resultados en csv por si se quieren explotar los datos en otra herramienta.

In [ ]:
x =min_df.xopt.array[0]
x0 = x[0]
x1 = x[1]
x2 = x[2]


global df, regressors_df, macros_df

s_w, s_m, s_y = min_df.season_s.ravel()[0], min_df.season_j.ravel()[0],min_df.season_k.ravel()[0]



print(df.head())
m = Prophet(growth=_model, seasonality_mode=_modality,
            holidays_prior_scale=x2,
            changepoint_range=x1,
            changepoint_prior_scale=x0,
            holidays=_holidays)

# Specifying seasonality:
if regressors_df:
    for metric_name in regressors_df.keys():
        m.add_regressor(metric_name)

if macros_df:
    for metric_name in macros_df.keys():
        m.add_regressor(metric_name)

m.add_country_holidays(country_name='ES')
m.add_seasonality(name='weekly', period=7, fourier_order=s_w)
m.add_seasonality(name='monthly', period=30.5, fourier_order=s_m)
m.add_seasonality(name='yearly', period=365.25, fourier_order=s_y)

m = m.fit(df)

future = m.make_future_dataframe(dias_prediccion)

future['ds'] = pd.to_datetime(future['ds'])
if regressors_df:
    for metric_df in regressors_df.values():
        future = pd.merge(future, metric_df, how='left', on='ds')

    

if macros_df:
    for metric_df in macros_df.values():
        future = pd.merge(future, metric_df, how='left', on='ds')


future['cap'] = df.y.quantile(.95)
future['floor'] = df.y.quantile(.5)

forecast = m.predict(future.fillna(0))

df_cv = cross_validation(m, initial=pd.Timedelta('365.25 days'),
                         horizon=pd.Timedelta(str(dias_prediccion) + ' days'))

df_p = performance_metrics(df_cv)

print(df_p.rmse.mean())
# Python
fig1 = m.plot(forecast)

fig2 = m.plot_components(forecast)

from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')

In [ ]:
pd.concat([forecast[['ds','yhat', 'yhat_lower', 'yhat_upper']],df.y], axis=1).to_csv('motos_android_forecast.csv', sep=';', index=False)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(forecast.covid_evolution)